# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/) and [Doxygen documentation](https://codedocs.xyz/CEED/libCEED/).

## Setting up libCEED for Python

Clone or download libCEED by running

In [ ]:
! git clone https://github.com/CEED/libCEED.git

Then move to the libCEED folder

In [ ]:
cd libCEED

And compile the base library by running

In [ ]:
! make

Build libCEED for Python by running

In [ ]:
python setup.py build install

Or without superuser permissions

In [ ]:
python setup.py build install --user

## CeedElemRestriction

Here we show some basic examples to illustrate the `CeedElemRestriction` class. In libCEED, a `CeedElemRestriction` groups the degrees of freedom (dofs) of the local vector according to the different elements they belong to (see [the API documentation](https://libceed.readthedocs.io/en/latest/libCEEDapi.html#finite-element-operator-decomposition)).

Here we illustrate the simple creation and application of a `CeedElemRestriction`, with user provided dof indices

In [2]:
import libceed
import numpy as np

# in this 1D example, the dofs are indexed as
# 
#  x --  x --  x --  x
# 10 -- 11 -- 12 -- 13

ceed = libceed.Ceed()

ne = 3

x = ceed.Vector(ne+1)
a = np.arange(10, 10 + ne+1, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

ind = np.zeros(2*ne, dtype="int32")
for i in range(ne):
  ind[2*i+0] = i
  ind[2*i+1] = i+1
    
r = ceed.ElemRestriction(ne, 2, ne+1, 1, ind, cmode=libceed.USE_POINTER)

y = ceed.Vector(2*ne)
y.set_value(0)

r.apply(x, y)

y_array = y.get_array_read()
print(y_array)
y.restore_array_read()

[10. 11. 11. 12. 12. 13.]


Note the repetition, in the output of the restriction, of the indices that are at the boundary between elements.

* In the following example, we illustrate the creation and use of a strided (identity) element restriction

In [5]:
# in this 1D example, the dofs are indexed as
# 
#  x --  x --  x --  x --  x --  x
# 10 -- 11 -- 12 -- 13 -- 14 -- 15

ne = 3

x = ceed.Vector(2*ne)
a = np.arange(10, 10 + 2*ne, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

strides = np.array([1, 2, 2], dtype="int32")

r = ceed.StridedElemRestriction(ne, 2, 2*ne, 1, strides)

y = ceed.Vector(2*ne)
y.set_value(0)

r.apply(x, y)

y_array = y.get_array_read()
print(y_array)
y.restore_array_read()

[10. 11. 12. 13. 14. 15.]


* In the following example, we illustrate the creation of a blocked element restriction (from an L-vector to an E-vector) and its transpose (inverse operation, from an E-vector to an L-vector)

In [3]:
# in this 1D example, the dofs are indexed as
# 
#  x --  x --  x --  x --  x --  x --  x --  x --  x
# 10 -- 11 -- 12 -- 13 -- 14 -- 15 -- 16 -- 17 -- 18

ne = 8
blksize = 5

x = ceed.Vector(ne+1)
a = np.arange(10, 10 + ne+1, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

ind = np.zeros(2*ne, dtype="int32")
for i in range(ne):
  ind[2*i+0] = i
  ind[2*i+1] = i+1

r = ceed.BlockedElemRestriction(ne, 2, blksize, ne+1, 1, ind,
                                cmode=libceed.USE_POINTER)

y = ceed.Vector(2*blksize*2)
y.set_value(0)

r.apply(x, y)

print(y)

x.set_value(0)
r.T.apply(y, x)

print(x)

CeedVector length 20
  10.000000
  11.000000
  12.000000
  13.000000
  14.000000
  11.000000
  12.000000
  13.000000
  14.000000
  15.000000
  15.000000
  16.000000
  17.000000
  17.000000
  17.000000
  16.000000
  17.000000
  18.000000
  18.000000
  18.000000

CeedVector length 9
  10.000000
  22.000000
  24.000000
  26.000000
  28.000000
  30.000000
  32.000000
  34.000000
  18.000000



* In the following example, we illustrate the creation and use of a blocked element restriction (from an L-vector to an E-vector) and its transpose (inverse operation, from an E-vector to an L-vector)

In [4]:
# in this 1D example, the dofs are indexed as
# 
#  x --  x --  x --  x --  x --  x --  x --  x --  x
# 10 -- 11 -- 12 -- 13 -- 14 -- 15 -- 16 -- 17 -- 18

ne = 8
blksize = 5

x = ceed.Vector(ne+1)
a = np.arange(10, 10 + ne+1, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

ind = np.zeros(2*ne, dtype="int32")
for i in range(ne):
  ind[2*i+0] = i
  ind[2*i+1] = i+1

r = ceed.BlockedElemRestriction(ne, 2, blksize, ne+1, 1, ind,
                                cmode=libceed.USE_POINTER)

y = ceed.Vector(blksize*2)
y.set_value(0)

r.apply_block(1, x, y)

print(y)

x.set_value(0)
r.T.apply_block(1, y, x)

print(x)

CeedVector length 10
  15.000000
  16.000000
  17.000000
  17.000000
  17.000000
  16.000000
  17.000000
  18.000000
  18.000000
  18.000000

CeedVector length 9
  0.000000
  0.000000
  0.000000
  0.000000
  0.000000
  15.000000
  32.000000
  34.000000
  18.000000



* In the following example, we illustrate how to extract the multiplicity of indices in an element restriction

In [7]:
# in this 1D example, there are four nodes per element
# 
#  x -- o -- o -- x -- o -- o -- x -- o -- o -- x

ne = 3

ind = np.zeros(4*ne, dtype="int32")

for i in range(ne):
  ind[4*i+0] = i*3+0
  ind[4*i+1] = i*3+1
  ind[4*i+2] = i*3+2
  ind[4*i+3] = i*3+3

r = ceed.ElemRestriction(ne, 4, 3*ne+1, 1, ind, cmode=libceed.USE_POINTER)

mult = r.get_multiplicity()

mult_array = mult.get_array_read()

print(mult_array)

mult.restore_array_read()

[1. 1. 1. 2. 1. 1. 2. 1. 1. 1.]


Note that the nodes at the boundary between elements have multiplicty 2, while the internal nodes and the outer boundary nodes, have multiplicuty 1.

* In the following example, we illustrate the creation and view of a blocked strided (identity) element restriction

In [10]:
# in this 1D example, there are three elements (four nodes in total) 
# 
#  x -- x -- x -- x

ne = 3

strides = np.array([1, 2, 2], dtype="int32")

r = ceed.BlockedStridedElemRestriction(ne, 2, 2, ne+1, 1, strides)

print(r)

Blocked CeedElemRestriction from (4, 1) to 3 elements with 2 nodes each and strides [1, 2, 2]

